### Import dataset

In [71]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

infri = pd.read_csv('data/infringement_dataset_v2.csv')

infri.fillna(value = -1,  
          inplace = True) 

display(infri)

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,first_name,last_name,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,Robert,Watkins,9251.77500,179055.000,179055.00,1.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,Jane,Navarro,56553.99000,435436.500,484191.00,3.0,0.0,0.0,0.0,3.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,David,Seagraves,5357.25000,24282.000,20106.00,1.0,0.0,0.0,0.0,1.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,Deborah,Tandy,23651.17500,272203.260,291695.50,5.0,3.0,1.0,0.0,9.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,David,Walker,12278.80500,150530.250,166638.75,6.0,0.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,Lawrence,Mcwayne,6605.91000,40455.000,40455.00,1.0,0.0,0.0,0.0,1.0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,Belinda,Loveland,10074.46500,57595.500,56821.50,1.0,0.0,0.0,0.0,1.0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,Aileen,Rasmussen,4770.40500,24162.750,20625.75,2.0,0.0,0.0,0.0,2.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,Mary,Marbury,10681.13250,121317.750,134439.75,2.0,0.0,0.0,0.0,2.0


### 2.2. and 2.3. for Correlation
Study the sensitivity <br>
Differential privacy mechanism

In [72]:
from matplotlib import pyplot as plt

df = infri[['infringed', 'past_avg_amount_annuity', 'past_avg_amt_application', 'past_avg_amt_credit', 'past_loans_approved', 'past_loans_canceled', 'past_loans_refused', 'past_loans_total', 'past_loans_unused']]
display(df)

,infringed,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_canceled,past_loans_refused,past_loans_total,past_loans_unused
0,1,9251.77500,179055.000,179055.00,1.0,0.0,0.0,1.0,0.0
1,0,56553.99000,435436.500,484191.00,3.0,0.0,0.0,3.0,0.0
2,0,5357.25000,24282.000,20106.00,1.0,0.0,0.0,1.0,0.0
3,0,23651.17500,272203.260,291695.50,5.0,1.0,3.0,9.0,0.0
4,0,12278.80500,150530.250,166638.75,6.0,0.0,0.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...
307506,0,6605.91000,40455.000,40455.00,1.0,0.0,0.0,1.0,0.0
307507,0,10074.46500,57595.500,56821.50,1.0,0.0,0.0,1.0,0.0
307508,0,4770.40500,24162.750,20625.75,2.0,0.0,0.0,2.0,0.0
307509,1,10681.13250,121317.750,134439.75,2.0,0.0,0.0,2.0,0.0


In [75]:
def add_laplace_noise(data):

  noise_scale = 1 / epsilon

  noisy_data = data + np.random.laplace(scale=noise_scale, size=data.shape)

  return noisy_data

In [77]:
def correlation_analysis(data):
  results = []

  for i in range(data.shape[1]):
    for j in range(i+1, data.shape[1]):
      r, p = stats.pearsonr(data.iloc[:, i], data.iloc[:, j])
      results.append(r)

  return results

results = correlation_analysis(df)

sensitivity_corr = 0

laplace_data = add_laplace_noise(df)

laplace_results = correlation_analysis(laplace_data)

change = 0

for r1, r2 in zip(laplace_results, results):
  change += abs(r1 - r2) 

sensitivity_corr = max(sensitivity_corr, change)

print("Sensitivity: ", sensitivity_corr)

Sensitivity:  7.401697876323568
